In [1]:
#필수 라이브러리 임포트
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd

C:\Users\USER\AppData\Local\Temp\ipykernel_4384\1188721120.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
!pip install tqdm

In [11]:
#크롬드라이버 설정
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument("--single-process")
# chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_experimental_option("detach", True)
#path='/home/sadf/chromedriver'
driver = webdriver.Chrome()


In [4]:
res = driver.get('https://m.place.naver.com/place/list?query=제주도관광지&level=top')
driver.implicitly_wait(20)

In [5]:
#2차 크롤링을 위한 bs4 셋팅
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])

session.mount('http://', HTTPAdapter(max_retries=retries))

In [6]:
from bs4 import BeautifulSoup
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')
soup = bs.select_one('div.YluNG')
naver_info = soup.select('li.VLTHu')

In [8]:
test_res = pd.read_csv('./place_list_all.csv')

In [9]:
test_res = test_res['Place ID'].tolist()

# 리스트의 각 원소에 대해 str() 메소드를 호출
for i in range(len(test_res)):
    test_res[i] = str(test_res[i])

In [24]:
test_res[19:20]

['12226192']

In [34]:
import pandas as pd
from tqdm import tqdm
# Create empty lists to store data
review_data = []
keyword_data = []
img_data = []
score_data = []
for place_ids in tqdm(test_res[19:20]):
    # 브라우저 열기
    driver = webdriver.Chrome()


    # URL 설정
    url = f"https://m.place.naver.com/place/{place_ids}/review/visitor?entry=ple"

    # URL로 이동
    driver.get(url)
    actions = driver.find_element(By.CSS_SELECTOR, 'body')
    actions.send_keys(Keys.END)
    driver.implicitly_wait(5)
    try:
        review_score_elements = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(5) > div> div.place_section.no_margin.mdJ86 > div > div > div.Xj_yJ > span.m7jAR.ohonc ")
        review_score = review_score_elements
        review_score = review_score.text if review_score else ''
        score_data.append({'place_ids': place_ids, 'review_score': review_score})
    except Exception as e:
            pass
    # keyword_content의 수를 구하기 위해 해당 CSS 선택자에 매치되는 요소들을 가져오기
    keyword_content_elements = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li > div.CsBE9 > span.nWiXa")

    # 가져온 요소들의 개수만큼 for 루프 돌리기
    for i in range(len(keyword_content_elements)):
        try:
            keyword_content = keyword_content_elements[i]
            keyword_count = driver.find_element(By.CSS_SELECTOR, f"#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li:nth-child({i+1}) > div.CsBE9 > span.TwM9q")
            # exception handling
            keyword_content = keyword_content.text if keyword_content else ''
            keyword_count = keyword_count.text.split('\n')[-1] if keyword_count else ''
        # Append data to the keyword_data list
            keyword_data.append({'place_ids': place_ids, 'keyword_content': keyword_content, 'keyword_count': keyword_count})
        except Exception as e:
            pass

    
    # 리뷰 더보기 버튼

    try:
        while True:
            more_button = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.NSTUp > div > a > span")
            if not more_button:
                break
            more_button[0].click()
            time.sleep(1)
    except Exception as e:
        pass
    #리뷰, 사진 , 날짜 태그 정보 가져오기
    try:
        review_content_elements = driver.find_elements(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div.place_section_content > ul > li > div.ZZ4OK > a > span.zPfVt')
        review_date_elements = driver.find_elements(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div > ul > li > div.qM6I7 > div > div._7kR3e > span:nth-child(1) > time')
        review_img_elements = driver.find_elements(By.CSS_SELECTOR,'#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k5tcc > div > ul > li > div.Mmpzx > div > div > div > div > a > img')
        
    except Exception as e:
        print('aaaaaa')
        pass
    # review for문
    for i in range(len(review_content_elements)):
        try:
            review_content = review_content_elements[i]
            review_date = review_date_elements[i]
            review_content = review_content.text if review_content else ''
            review_date = review_date.text if review_date else ''
            # Append data to the review_data list
            review_data.append({'place_ids': place_ids, 'review_content': review_content, 'review_date': review_date})
        except Exception as e:
            pass
    # img for문
    for i in range(len(review_img_elements)):
        try:
            review_img = review_img_elements[i]
            review_img = review_img.get_attribute("src") if review_img else ''
            img_data.append({'place_ids': place_ids,'review_image' : review_img})
        except Exception as e:
            pass


    time.sleep(0.2)
    
# Create data frames
keyword_df = pd.DataFrame(keyword_data)
review_df = pd.DataFrame(review_data)
img_df = pd.DataFrame(img_data)
score_df = pd.DataFrame(score_data)

100%|██████████| 1/1 [00:10<00:00, 10.41s/it]


In [35]:
review_df 

,place_ids,review_content,review_date
0,12226192,"새롭게 공사했다해서 방문한 주상절리! 제주도민은 무료이고, 주차비만 지불하면 돼요!...",1.13.토
1,12226192,주상절리 보고싶어서 방문했는데 유도선도 잘 되어있고 테크도 잘 만들어져 있어서 가볼...,23.12.27.수
2,12226192,언제봐도 경관이 멋진 대포주상절리. 절리의 독특한 육각기둥형태는 학문적으로 참 의미...,1.7.일
3,12226192,걸어다닐 수 있는 데크가 잘 되어 있어요. 최근에 바꾼 듯한 느낌이 들고 주상절리를...,23.12.3.일
4,12226192,"풍경이 너무 멋지고, 바다의 압도적인 풍경이 너무 근사하고 좋았어요! 입장료가 있지...",23.11.19.일
5,12226192,언제와도 멋진 경관의 주상절리 입니다. 주차장도 넓고 편의시설이 잘 갖춰져있어 이용...,1.13.토
6,12226192,사람이 별로 없었지만 타이밍이 좋았던것 같고 단체로 많이 오시는 코스인가봅니다. 사...,23.11.28.화
7,12226192,"너무 아릅답고 좋았던 곳입니다.\n입장료 2천원, 주차비 2천원 냈고\n30분정도 ...",23.12.30.토
8,12226192,작년에는 공사중이라 못 들어갔는데\n지금 뷰가 완전 끝내줍니다 ㅎㅎ\n제주 오션 뷰...,23.12.15.금
9,12226192,제주 대포주상절리에 다녀왔습니다. 공사 후 다녀왔는데요. 전망대가 잘 꾸며져 있어 ...,23.9.30.토


In [36]:
keyword_df

,place_ids,keyword_content,keyword_count
0,12226192,"""사진이 잘 나와요""",165
1,12226192,"""주차하기 편해요""",80
2,12226192,"""볼거리가 많아요""",74
3,12226192,"""산책로가 잘 되어있어요""",64
4,12226192,"""붐비지 않아요""",43
